In [6]:
from pathlib import Path
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.feather as feather
import os
import pickle

In [3]:
def convert_mixed_types(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            # Convert mixed types to string (or another consistent type)
            df[col] = df[col].astype(str)
    return df

In [ ]:
writer = None
schema = None
original_schema = None
csv_file = '/mnt/qdata/rawdata/UKBIOBANK/baskets/4053862/ukb677731.csv'
parquet_file = 'temp_cols.parquet'
schema_file= 'schema.pickle'

if os.path.exists(parquet_file):
    os.remove(parquet_file)

full_df = pd.read_csv(csv_file, nrows=10)
original_schema = pa.Schema.from_pandas(full_df)

with open(schema_file, 'wb') as f:
    pickle.dump(original_schema, f)

cols = list(full_df.columns)

for i in range(0, len(cols), 1000):
    subset_df = pd.read_csv(csv_file, usecols=cols[i:i+1000])
    subset_df = convert_mixed_types(subset_df)
    subset_table = pa.Table.from_pandas(subset_df)
    
    if i == 0:
        writer = pq.ParquetWriter(parquet_file, subset_table.schema)

    writer.write_table(subset_table)

writer.close()
